In [13]:
import importlib
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
import pandas as pd
import scipy.stats
import seaborn as sns
from scipy.special import expit
from sklearn import linear_model, metrics
import matplotlib.lines as mlines
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import auc, plot_roc_curve, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, StratifiedKFold

from mapra import prep
from fopra import fopra_driver

sns.set_theme(style='white')
rng = np.random.default_rng(12345)
WD = Path('.').resolve().parent

In [14]:
importlib.reload(prep)
importlib.reload(fopra_driver)
data = prep.dataset()
df = data.dataframe_abbrev(reduced=True)

test_size = .2
seed = 35

splits = data.uniprot_train_test_split(df=df, test_size=.2, random_state=seed)
fopra_driver.seeds

/home/quirin/PYTHON/mapra


/home/quirin/PYTHON/mapra/mapra/prep.py:42: RuntimeWarning: unexpected FASTA filename: /home/quirin/PYTHON/mapra/all_sequences_prothermdb.fasta
  warnings.warn('unexpected FASTA filename: ' + str(fasta), RuntimeWarning)
/home/quirin/PYTHON/mapra/mapra/prep.py:53: UserWarning: found not exactly one TSV with annotations:
/home/quirin/PYTHON/mapra/wildtype_pHs.tsv
/home/quirin/PYTHON/mapra/prothermdb_annotations.tsv
/home/quirin/PYTHON/mapra/wildtypes.tsv
/home/quirin/PYTHON/mapra/txts/uniprot_to_pdb.tsv
/home/quirin/PYTHON/mapra/txts/uniprot_to_pdb_unmapped.tsv
/home/quirin/PYTHON/mapra/full_dbs/crop.tsv
/home/quirin/PYTHON/mapra/full_dbs/wild_types.tsv
/home/quirin/PYTHON/mapra/full_dbs/wildtypes.tsv
/home/quirin/PYTHON/mapra/full_dbs/Tm_dataset.tsv
/home/quirin/PYTHON/mapra/CAGI6/cam_predictions.tsv
  warnings.warn('found not exactly one TSV with annotations:\n' + '\n'.join(str(tsv) for tsv in tsvs))


loading from /home/quirin/PYTHON/mapra/pkl/h5_slice_0.pkl
split 35 targeted 0.2, real test sizes: 0.2020:0.2113:0.2057, record test sizes: 0.3623:0.1307:0.0980


[0, 1, 2, 3, 4, 5, 6, 1024, 1025, 1026, 1027, 1028, 1029]

In [15]:
ar = np.load(str(WD / 'txts' / f'fopra_all.npy'),
             allow_pickle=False)



array([[0.00000000e+00, 1.88000000e+02, 4.80000000e+01, ...,
        2.02020202e-01, 2.60494311e-01, 0.00000000e+00],
       [1.00000000e+00, 8.10000000e+01, 1.40000000e+01, ...,
        2.11267606e-01, 1.66752445e-01, 0.00000000e+00],
       [2.00000000e+00, 2.16000000e+02, 4.90000000e+01, ...,
        2.05673759e-01, 1.41468254e-01, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.18500000e+03, 1.02400000e+03, ...,
        2.02020202e-01, 2.20870930e-01, 1.02900000e+03],
       [1.00000000e+00, 9.74000000e+02, 1.02400000e+03, ...,
        2.11267606e-01, 8.85229027e-02, 1.02900000e+03],
       [2.00000000e+00, 1.44400000e+03, 1.02400000e+03, ...,
        2.05673759e-01, 4.79166667e-01, 1.02900000e+03]])